In [162]:
import numpy as np
from copy import deepcopy

from keras.models import load_model
import tictactoe_env

import importlib
importlib.reload(tictactoe_env)

<module 'tictactoe_env' from '/Users/techmu/Documents/pj_2019-b3-new-display/ai-tictactoe/play_dev/tictactoe_env.py'>

In [163]:
# 学習済みモデルを読み込み
model = load_model('../train/my_model.h5')

In [174]:
# OXゲームクラスをインスタンス化
env = tictactoe_env.TictactoeEnv()

In [165]:
# おけるマスの座標をリストアップ
can_put_idx = [idx for idx, cell in enumerate(env.board) if cell == env.E_ID]

can_put_idx

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [166]:
# 置けるパターンを全て格納
pred_boards = []
for idx in can_put_idx:
    board = deepcopy(env.board)
    board[idx] = env.current_player
    pred_boards.append(board)
pred_boards = np.array(pred_boards)
pred_boards

array([[-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1]], dtype=int32)

In [167]:
# 最も勝てる確率の高いパターンを決定
pred = model.predict(pred_board)
np.argmax([p[0] for p in pred])

4

In [168]:
pred

array([[1.5380043e-13, 1.5458373e-03, 9.9845421e-01],
       [1.1806880e-19, 2.1600226e-07, 9.9999976e-01],
       [5.4162741e-17, 1.9765864e-06, 9.9999797e-01],
       [2.5083810e-18, 4.4512581e-07, 9.9999952e-01],
       [2.3593795e-13, 1.3806598e-03, 9.9861932e-01],
       [7.1632840e-18, 1.2440331e-06, 9.9999881e-01],
       [8.4628655e-16, 7.6754848e-05, 9.9992323e-01],
       [2.9716727e-16, 7.3516290e-05, 9.9992645e-01],
       [2.2495728e-15, 1.3460254e-04, 9.9986541e-01]], dtype=float32)

In [144]:
pred[1].sum()

1.0

In [159]:
def cpu(env):
    # 自分が勝てる確率の閾値
    TH_PRED_P_WIN = 0.05
    
    # おけるマスの座標をリストアップ
    can_put_idx = [idx for idx, cell in enumerate(env.board) if cell == env.E_ID]
    
    # 置けるパターンを全て格納
    pred_boards = []
    for idx in can_put_idx:
        board = deepcopy(env.board)
        board[idx] = env.current_player
        pred_boards.append(board)
    pred_boards = np.array(pred_boards)

    # 各パターンで勝敗の確率を計算
    pred = model.predict(pred_board)
    
    # 自分が一番勝てる確率と敵が一番負ける確率が高いパターンをそれぞれ決定
    # - (x, -, y) = (-1, 0, 1) に+1して添字に利用
    pred_p_id = env.current_player + 1
    pred_e_id = -env.current_player + 1
    best_p_win_pred = np.argmax([p[pred_p_id] for p in pred])
    best_e_lose_pred = np.argmax([p[pred_e_id] for p in pred])
    
    # パターンを決定
    if pred_p_id >= TH_PRED_P_WIN:
        # 自分が一番勝てる確率が閾値以上ならそうする
        best_pred = best_p_win_pred
    else:
        # 閾値未満なら、敵が一番負けるようにする
        best_pred = best_e_lose_pred
    
    # 決定したパターンで置く座標を返す
    return can_put_idx[best_pred]

In [169]:
env.put(cpu(env))

env.board.reshape(3, 3)

array([[ 0, -1,  0],
       [ 0,  0,  0],
       [ 0,  0,  0]], dtype=int32)

In [170]:
env.put(4)

env.board.reshape(3, 3)

array([[ 0, -1,  0],
       [ 0,  1,  0],
       [ 0,  0,  0]], dtype=int32)

In [171]:
env.put(cpu(env))

env.board.reshape(3, 3)

array([[ 0, -1, -1],
       [ 0,  1,  0],
       [ 0,  0,  0]], dtype=int32)

In [172]:
env.put(0)

env.board.reshape(3, 3)

array([[ 1, -1, -1],
       [ 0,  1,  0],
       [ 0,  0,  0]], dtype=int32)

In [173]:
env.put(cpu(env))

env.board.reshape(3, 3)

array([[ 1, -1, -1],
       [ 0,  1, -1],
       [ 0,  0,  0]], dtype=int32)